In [1]:
# Order preparation for SnP

# STATUS: Incomplete
# Run-time: 10 seconds

# Dependencies:
# /zdata/pkls/*.pkl - for pickles generated by 01_snp_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 1300, clientId=11)

In [48]:
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math

#...assignments
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.05       # Upper or Lower base multiple for prices

min_rom = 0.8
min_pop = 0.85

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

#...get current positions
#________________________

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

#...identify Stock and Option
rights = [s.right for s in df_p.contract]
df_p['Type'] = ['Stock' if r == '0' else 'Option' for r in rights]

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# harvest option Dataframe
df_hop = df_p[df_p.Type == 'Option'].reset_index(drop=True)

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_hop.contract]

df_hop['dte'] = [(util.parseIBDatetime(d.lastTradeDateOrContractMonth) - datetime.datetime.now().date()).days for d in df_hop.contract]

discount = [m for m in map(expPricePct, expiry)]
df_hop['hvstPrice'] = pd.concat([df_hop.averageCost*discount, 
                               df_hop.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_hop.hvstPrice = np.floor(df_hop.hvstPrice/base)*base # round down to the nearest 0.05

df_hop.loc[df_hop.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
df_hop['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_hop.position, df_hop.hvstPrice)]

# ignore data for dte < 3 days. These are as good as gone.
df_h = df_hop[df_hop.dte > 3].reset_index(drop=True)

hqc = ib.qualifyContracts(*df_h.contract)
df_h = df_h.assign(qual_contract=hqc)


ValueError: time data '' does not match format '%Y%m%d  %H:%M:%S'

In [49]:
# ...sowing prepration
#_____________________

# #... get the lots and margins
# # from 5paisa
# paisaurl = "https://www.5paisa.com/5pit/spma.asp"
# df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# # Rename Symbol and Margin fields
# df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# # Convert columns to numeric and make margin to pct
# df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
# df_paisa.marginpct = df_paisa.marginpct.div(100)

# # Truncate to 9 characters for ibSymbol
# df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# # nseSymbol to ibSymbol dictionary for conversion
# ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# # remap ibSymbol, based on the dictionary
# df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])


#... find margins
orders = [Order(action=np.where(q<0, 'SELL', 'BUY'), totalQuantity=abs(q), orderType='MKT') for q in df_p.position]

cs = list(df_p.contract)
qc = ib.qualifyContracts(*cs)
df_p = df_p.assign(margin=[float(ib.whatIfOrder(c, o).initMarginChange) for c, o in zip(qc, orders)])


#... make the blacklist
df1 = df_p.assign(lot=np.where(df_p.Type == 'Option', 100*df_p.position, df_p.position))
df1 = df1.groupby('ibSymbol').sum()
df1 = df1.assign(TotMgnPerShr = df1.margin/df1.position)

df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.used_margin + df1.max_margin

In [50]:
df1['max_units'] = (df1.avail_margin/(df1.lot*df1.TotMgnPerShr)).apply(np.floor)

In [51]:
df1

,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,margin,lot,TotMgnPerShr,used_margin,max_margin,avail_margin,max_units
ibSymbol,,,,,,,,,,,,,
AXP,99.0,102.950396,10274.96,110.833500,538.29,0.0,3240.42,9900.0,32.731515,-3240.42,8251.5186,5011.0986,0.0
FDX,297.0,179.421237,53753.63,215.133467,-5443.47,0.0,16931.18,29700.0,57.007340,-16931.18,8251.5186,-8679.6614,-1.0


In [ ]:
blacklist = list(df1[df1.max_units <= 0].index)

#...build the high-pop-roc dataframe
fs = listdir('./zdata/')

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

df_opt = pd.concat([pd.read_pickle('./zdata/'+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle('./zdata/'+f).reset_index() for f in ohlcs], axis=0, sort=True)

df_und = pd.concat([pd.read_pickle('./zdata/'+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# take only high rom and pops
df_opt1 = df_opt[(df_opt.rom > min_rom) & 
                   (df_opt.dte > 2) &
                   (df_opt['pop'] > min_pop)]

In [ ]:
df_mlot = df_und[['symbol', 'lot']]

df_opt2 = df_opt1.merge(df_mlot, on='symbol', how='inner')

df_opt2['expPrice'] = round(round((df_opt2.price+0.1)/ base) * base, -int(math.floor(math.log10(base))))